In [1]:
from vpython import *
from queue import PriorityQueue 

<IPython.core.display.Javascript object>

## A. Define Logic Gates

In [2]:
# Wires
class Net:
    def __init__(self, id, curve):
        self.val = 0.5
        self.id = id
        self.trigger = False   # indicates whether net has been updated or not
        self.curve = curve
    
    # Setters
    def set_val(self, val):
        self.val = val
        self.trigger = True
    
    # Getters
    def get_val(self):
        return self.val
    
    def get_id(self):
        return self.id
    
    def get_trigger(self):
        return self.trigger

    def turnoff_trigger(self):
        self.trigger = False
        
    def get_curve(self):
        return self.curve
    
class Gate:
    def __init__(self, net_in1, net_in2, net_out, gate_id, delay, time, ev):
        self._in_net1 = net_in1
        self._in_net2 = net_in2
        self.out_net = net_out  # output
        
        self._output = 0.5
        self._id = gate_id
        self.time = time
        self.delay = delay
        self.evaluate_function = ev  # function that tells the Gate what to do
    
    def evaluate(self):
        self.evaluate_function(self._in_net1, self._in_net2, self.out_net)
        
    # Update value of 2 inputs
    def update_in_nets(self, val_1, val_2):        
        self._in_net1.set_val(val_1)
        self._in_net2.set_val(val_2)
        
    def get_id(self):
        return self._id
                
    def get_delay(self):
        return self.delay
                
    def get_time(self):
        return self.time
    
    def is_updated(self):
        return self._in_net1.get_trigger() or self._in_net2.get_trigger()
    
    def set_time(self, time):
        self.time = time
    
    # Turn off the trigger once update is applied
    def in_net_update_applied(self):
        self._in_net1.turnoff_trigger()
        self._in_net2.turnoff_trigger()
        
    def out_net_update_applied(self):
        self.out_net.turnoff_trigger()
        
    def get_in_net1(self):
        return self._in_net1
    
    def get_in_net2(self):
        return self._in_net2
    
    def get_out_net(self):
        return self.out_net
    
    def draw_curve_before(self):
        self._in_net1.get_curve().plot(self.time - self.delay, self._in_net1.get_val())
        self._in_net2.get_curve().plot(self.time - self.delay, self._in_net2.get_val())
        self.out_net.get_curve().plot(self.time, self.out_net.get_val())
    
    def draw_curve_after(self):
        self._in_net1.get_curve().plot(self.time, self._in_net1.get_val())
        self._in_net2.get_curve().plot(self.time, self._in_net2.get_val())
        self.out_net.get_curve().plot(self.time + self.delay, self.out_net.get_val())
        
    def draw_curve_normal(self):
        self._in_net1.get_curve().plot(self.time, self._in_net1.get_val())
        self._in_net2.get_curve().plot(self.time, self._in_net2.get_val())
        self.out_net.get_curve().plot(self.time, self.out_net.get_val())
        
    def draw_in_net1(self):
        self._in_net1.get_curve().plot(self.time, self._in_net1.get_val())
        
    def draw_in_net2(self):
        self._in_net2.get_curve().plot(self.time, self._in_net2.get_val())
    
    def draw_out_net(self):
        self.out_net.get_curve().plot(self.time, self.out_net.get_val())
    
# Event of gate receiving different input values
class Event:
    def __init__(self, gate, time):
        self.gate = gate
        self.time = time
    
    def __lt__(self, obj):
        """self < obj."""
        return self.time < obj.get_time()

    def __le__(self, obj):
        """self <= obj."""
        return self.time <= obj.get_time()

    def __eq__(self, obj):
        """self == obj."""
        return self.time == obj.get_time()

    def __ne__(self, obj):
        """self != obj."""
        return self.time != obj.get_time()

    def __gt__(self, obj):
        """self > obj."""
        return self.time > obj.get_time()

    def __ge__(self, obj):
        """self >= obj."""
        return self.time >= obj.get_time()
    
    def get_gate(self):
        return self.gate
    
    def get_time(self):
        return self.time
    
def evaluate_NAND(in1, in2, out):
    out.set_val(int(not(in1.get_val() and in2.get_val()) == True))  # Convert bool to int

def evaluate_NOR(in1, in2, out):
    out.set_val(int(not(in1.get_val() or in2.get_val()) == True))
    
def evaluate_NOT(in1, in2, out):
    out.set_val(int(not in1.get_val()))


## B. Confirm Logic Gates by Testing  /  C. Hold Events in Sequence using Priority Queue

I just decided to implement part B using part C's Event class. Event class can be found from above.  
**Note:** I don't know why, but graphs only get displayed below the top-most cell, so be aware of that.

#### Visual Representation using gcurve

In [3]:
def simulation_single_gate(gate_type):

    t = 0
    delay = 0.05
    
    # Tasks that we want to use to test our gate
    # Repeat 0,0 0,1 1,0 1,1 cycle twice
    task = [(0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1)]
    
    # Graphs
    g1 = graph(title='Curve of net A for {}'.format(gate_type), xtitle='time', ytitle='bit', xmin=0, ymin=0)
    g2 = graph(title='Curve of net B for {}'.format(gate_type), xtitle='time', ytitle='bit', xmin=0, ymin=0)
    g3 = graph(title='Curve of net Q for {}'.format(gate_type), xtitle='time', ytitle='bit', xmin=0, ymin=0)
    
    # Curves w/ associated graphs
    curve1 = gcurve(color=color.black, graph=g1)
    curve2 = gcurve(color=color.blue, graph=g2)
    curve3 = gcurve(color=color.red, graph=g3)
    
    fn = None
    if gate_type == "NAND":
        fn = evaluate_NAND
    elif gate_type == "NOR":
        fn = evaluate_NOR
    
    net_in1 = Net(1, curve1)
    net_in2 = Net(2, curve2)
    net_out = Net(3, curve3)
    q = PriorityQueue()
    q.put(Event(Gate(net_in1, net_in2, net_out, 1, delay, t, fn), t))  # Given delay: 0.05
    
    while not q.empty():
        val_1 = task[0][0]
        val_2 = task[0][1]
        task.pop(0)
        
        event = q.get()
        gate = event.get_gate()
        
        # Update time -> draw curve before -> update actual net values -> draw curve after
        gate.set_time(t)
        gate.draw_curve_before()
        gate.update_in_nets(val_1, val_2)
        gate.evaluate()
        gate.draw_curve_after()
        
        # Update Time
        t += 1
        
        # Check if we still have test tasks. If we do, add event
        # If any of in-nets are updated, then add event
        if len(task):
            q.put(Event(gate, t))
        

In [4]:
simulation_single_gate("NAND")
simulation_single_gate("NOR")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Analysis:**  
I verified the result by checking the input & output graphs of each gate. By using online truth tables for each gate type, I was able to confirm that the simulation is working as intended.

## D. Simple Combinational Network of gates

In [5]:
def simulation_comb_network(gate_type):
    
    t = 0
    delay = 0.05
    
    # *********
    # * Tasks *
    # *********
    # Repeat 0,0 0,1 1,0 1,1 cycle twice
    task1 = [(0,0), (0,1), (1,0), (1,1), (0,0), (0,1), (1,0), (1,1)]
    task2 = [0, 0, 0, 0, 1, 1, 1, 1]
    
    # **********
    # * Graphs *
    # **********
    # 1st Gate
    g1 = graph(title='Curve of input net 1 of 1st Gate for {}'.format(gate_type), xtitle='time', ytitle='bit', xmin=0, ymin=0)
    g2 = graph(title='Curve of input net 2 of 1st Gate for {}'.format(gate_type), xtitle='time', ytitle='bit', xmin=0, ymin=0)
    g4 = graph(title='Curve of output net of 1st Gate for {}'.format(gate_type), xtitle='time', ytitle='bit', xmin=0, ymin=0)
    
    # 2nd Gate
    g3 = graph(title='Curve of two input nets of 2nd Gate for {}'.format(gate_type), xtitle='time', ytitle='bit', xmin=0, ymin=0)
    g5 = graph(title='Curve of output net of 2nd Gate for {}'.format(gate_type), xtitle='time', ytitle='bit', xmin=0, ymin=0)

    # 3rd Gate
    g6 = graph(title='Curve of output of 3rd Gate for {}'.format(gate_type), xtitle='time', ytitle='bit', xmin=0, ymin=0)

    # *********************************
    # ** Curves w/ associated graphs **
    # *********************************
    curve1 = gcurve(color=color.black, graph=g1)
    curve2 = gcurve(color=color.blue, graph=g2)
    curve4 = gcurve(color=color.red, graph=g4)
    
    curve3 = gcurve(color=color.orange, graph=g3)
    curve5 = gcurve(color=color.cyan, graph=g5)
    
    curve6 = gcurve(color=color.green, graph=g6)   
    
    fn = None
    if gate_type == "NAND":
        fn = evaluate_NAND
    elif gate_type == "NOR":
        fn = evaluate_NOR
    
    # *********************
    # * Network hardcoded *
    # *********************
    
    # Curve attached to net for ease of graphing
    net_in1 = Net(1, curve1)
    net_in2 = Net(2, curve2)
    net_in3 = Net(3, curve3)
    net_out4 = Net(4, curve4)
    net_out5 = Net(5, curve5)
    net_out6 = Net(6, curve6)
    
    gate1 = Gate(net_in1, net_in2, net_out4, 1, delay, t, fn)  # Given delay: 0.05
    gate2 = Gate(net_in3, net_in3, net_out5, 2, delay, t, fn)
    gate3 = Gate(net_out4, net_out5, net_out6, 3, delay, t, fn)
    
    # Draw the beginning part before the loop runs to make the visual better
    gate1.draw_curve_normal()
    gate2.draw_curve_normal()
    gate3.draw_curve_normal()
    
    q = PriorityQueue()
    q.put(Event(gate1, t))   
    q.put(Event(gate2, t))   
    q.put(Event(gate3, t))      
    
    # *******
    # * Run *
    # *******
    while not q.empty():

        # Grab the event from Priority queue
        event = q.get()
        gate = event.get_gate()
        
        # Process: update time -> draw curve before -> update actual net values -> draw curve after
        
        # Update time
        gate.set_time(gate.get_time() + 1)  # Update each gate's unique time
            
        # Draw curve before values get updated
        gate.draw_curve_before()
            
        # Depends on the gate's id, update values accordingly
        if gate.get_id() == 1:
            val_1 = task1[0][0]
            val_2 = task1[0][1]
            task1.pop(0)
            gate.update_in_nets(val_1, val_2)
                        
        elif gate.get_id() == 2:
            val_1 = task2[0]
            task2.pop(0)
            gate.update_in_nets(val_1, val_1)
                           
        # Update gate's nets using the newly added values
        gate.evaluate()
        
        # Draw the curve again to have square wave
        gate.draw_curve_after()
        
        # Handle gate to determine whether the gate should get into the Priority Queue
        # For gate id=1 & 2
        if (gate.get_id() == 1 and len(task1)) or (gate.get_id() == 2 and len(task2)):
            gate.in_net_update_applied()
            q.put(Event(gate, gate.get_time()))
            
        # Handle gate id=3
        if gate.get_id() == 3 and (len(task1) or len(task2)):
            gate.in_net_update_applied()
            q.put(Event(gate, gate.get_time()))
        

In [6]:
simulation_comb_network("NAND")
simulation_comb_network("NOR")

**How I verified:**  
As instructed, I displayed the values of all the nets.  

I've verified the result by making a truth table of the entire network and keeping a track of how each bit is supposed to change over time. In the end, I was able to confirm that the graphs that above simulations have created are accurate.

## E. Simulation of NAND gate D-Latch

In [7]:
def draw_before_update_D_Latch(gate):
    if gate.get_id() == 1:
        gate.get_in_net1().get_curve().plot(gate.get_time() - gate.get_delay(), gate.get_in_net1().get_val())
        gate.get_in_net2().get_curve().plot(gate.get_time() - gate.get_delay(),  gate.get_in_net2().get_val())
    elif gate.get_id() == 4:
        gate.get_out_net().get_curve().plot(gate.get_time(), gate.get_out_net().get_val())
    elif gate.get_id() == 5:
        gate.get_out_net().get_curve().plot(gate.get_time(), gate.get_out_net().get_val())
    
def draw_after_update_D_Latch(gate):
    if gate.get_id() == 1:
        gate.get_in_net1().get_curve().plot(gate.get_time(), gate.get_in_net1().get_val())
        gate.get_in_net2().get_curve().plot(gate.get_time(),  gate.get_in_net2().get_val())
    elif gate.get_id() == 4:
        gate.get_out_net().get_curve().plot(gate.get_time() + gate.get_delay(), gate.get_out_net().get_val())
    elif gate.get_id() == 5:
        gate.get_out_net().get_curve().plot(gate.get_time() + gate.get_delay(), gate.get_out_net().get_val())

def simulation_d_latch():
    
    t = 0
    delay = 0.05
    
    # *********
    # * Tasks *
    # *********
    # Repeat 0,0 0,1 1,0 1,1 cycle twice
    task1 = [0, 0, 0, 0, 1, 1, 1, 1]  # D
    task2 = [0, 1, 0, 1, 0, 1, 0, 1]   # CLK
    
    # **********
    # * Graphs *
    # **********
    # 1st Gate
    g1 = graph(title='D input of D-Latch', xtitle='time', ytitle='bit', xmin=0, ymin=0)
    g2 = graph(title='CLK of D-Latch', xtitle='time', ytitle='bit', xmin=0, ymin=0)
    g3 = graph(title='Q of D-Latch', xtitle='time', ytitle='bit', xmin=0, ymin=0)
    g4 = graph(title="Q' of D-Latch", xtitle='time', ytitle='bit', xmin=0, ymin=0)

    # *********************************
    # ** Curves w/ associated graphs **
    # *********************************
    curve1 = gcurve(color=color.black, graph=g1)  # D
    curve2 = gcurve(color=color.magenta, graph=g2)   # CLK
    curve3 = gcurve(color=color.orange, graph=g3)    # Q
    curve4 = gcurve(color=color.purple, graph=g4)  # Q'
    
    # *********************
    # * Network hardcoded *
    # *********************
    
    # Curve attached to net for ease of graphing
    net_D = Net(1, curve1)
    net_CLK = Net(2, curve2)
    net_out_NAND1 = Net(4, None)
    
    net_inverted = Net(3, None)
    net_out_NAND2 = Net(5, None)
    
    net_Q = Net(6, curve3)
    net_Q_inv = Net(7, curve4)
    
    # Set Q to be 1 by default for ease of debugging
    net_Q.set_val(1)
    
    # Gates
    # Given delay: 0.05
    gate1 = Gate(net_D, net_CLK, net_out_NAND1, 1, delay, t, evaluate_NAND)   # 1st NAND
    gate2 = Gate(net_D, Net(0, None), net_inverted, 2, 0, t, evaluate_NOT)   # NOT
    gate3 = Gate(net_CLK, net_inverted, net_out_NAND2, 3, delay, t, evaluate_NAND) # 2nd NAND
    gate4 = Gate(net_out_NAND1, net_Q_inv, net_Q, 4, delay*2, t, evaluate_NAND) # 3rd NAND
    gate5 = Gate(net_out_NAND2, net_Q, net_Q_inv, 5, delay*2, t, evaluate_NAND) # 4th NAND
    
    # Draw the beginning part before the loop runs to make the visual better
    gate1.draw_in_net1()
    gate1.draw_in_net2()
    gate4.draw_out_net()
    gate5.draw_out_net()
    
    q = PriorityQueue()
    q.put(Event(gate1, t))   
    q.put(Event(gate2, t))   
    q.put(Event(gate3, t))      
    q.put(Event(gate4, t))   
    q.put(Event(gate5, t))   
    
    # *******
    # * Run *
    # *******
    while not q.empty():

        # Grab the event from Priority queue
        event = q.get()
        gate = event.get_gate()
        
        # Process: update time -> draw curve before -> update actual net values -> draw curve after
        
        # Update time
        gate.set_time(gate.get_time() + 1)  # Update each gate's unique time
            
        # Draw curve before values get updated
        draw_before_update_D_Latch(gate)
            
        # Depends on the gate's id, update values accordingly
        if gate.get_id() == 1:
            # Since I do not want it to run forever, have a way to stop it (in this case, when all the D input tasks are given)
            if len(task1) == 0: break
            gate.draw_in_net1()
            gate.draw_in_net2()
            val_1 = task1[0]
            val_2 = task2[0]
            gate.update_in_nets(val_1, val_2)            
                        
        elif gate.get_id() == 3:
            val_1 = task2[0]
            task1.pop(0)
            task2.pop(0)
            gate.update_in_nets(val_1, val_1)
                           
        # Update gate's nets using the newly added values
        gate.evaluate()
        
        # Draw the curve again to have square wave
        draw_after_update_D_Latch(gate)
        
        if (gate.is_updated()):
            q.put(Event(gate, gate.get_time()))
            gate.in_net_update_applied()
        

In [8]:
simulation_d_latch()

## F. Comment on Experience / Results

**Analysis:**  
Since there are too many nets, I decided to only display D, CLK, Q, and Q'   
  
In the simulation, I successfully created a D-Latch and displayed the D and CLK(EN) inputs, as well as the Q and Q' outputs, in the form of a graph. Regarding verification, I confirmed the results using an online website that allows users to build circuits and monitor changes in bits as they progress.   
  
In terms of creating combinational versus sequential circuits, there was a significant difference. The design of the combinational circuit was straightforward since its input does not depend on the output. However, for the sequential circuit, I had to consider this dependency and create a net shared by both the input and output. This ensured that whenever the output changed, the value of the net also changed, providing feedback to the network as an input.   
  
In terms of simulating and verifying the legitimacy of the simulation, I found testing the sequential circuit much more challenging than the combinational one. With multiple inputs and outputs changing simultaneously, it was difficult to keep track of them all. However, in the end, I was able to verify both the combinational and sequential simulations.